<a href="https://colab.research.google.com/github/yoonkim313/dataCampusProject-Team10/blob/master/Title%20Extraction/summarization_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [193]:
!pip install kss

  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251546 sha256=7941da6fe761bc4583fc77bd3d5a42aeca51a4bb1483dfe3643ad1f7c5c0eb24
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
Successfully built kss


In [280]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import kss
import re

In [292]:
alphabets = "([A-Za-z가-힣])"
numbers = "^\d*[.,]?"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n", " ")
    text = re.sub(websites, "<prd>\\1", text)
    if "Ph.D" in text: text = text.replace("Ph.D.", "Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] ", " \\1<prd> ", text)
    text = re.sub(acronyms + " " + starters, "\\1<stop> \\2", text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]", "\\1<prd>\\2<prd>\\3<prd>",
                  text)
    text = re.sub(numbers + "[.]" + " ", " \\1<prd> ", text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]", "\\1<prd>\\2<prd>", text)
    text = re.sub(alphabets + "[.]" + alphabets, "\\1<prd>\\2", text)
    text = re.sub(" " + suffixes + "[.]", " \\1<prd>", text)
    text = re.sub(" " + alphabets + "[.]", " \\1<prd>", text)
    if "!" in text: text = text.replace("!\"", "\"!")
    if "?" in text: text = text.replace("?\"", "\"?")
    text = text.replace(".", ".<stop>")
    text = text.replace("?", "?<stop>")
    text = text.replace("!", "!<stop>")
    text = text.replace("<prd>", ".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

def read_data(PATH, file_name):
  df = pd.read_excel(os.path.join(PATH,file_name),encoding='utf-sig-8',na_filter=True)
  return df

def processScript(df):
  sentences=[]
  label=[]
  errors=[]
  sentence_length=[]
  for _ in range(len(df)):
      target = df.label[_]
      temp = kss.split_sentences(df.data[_])
      l=[]
      try:
         print("split sentences",len(temp),"target length",len(literal_eval(target)))
         if len(temp) == len(literal_eval(target)):
            sentences.extend(temp)
            label.extend(literal_eval(target))
         else:
           print(f"🤗 error : length unmatched {_}")
           sentences.extend(temp)
           l = ["fix" for _ in temp]
           print("LENGTH",len(l))
           label.extend(l)
           errors.append(_)
           sentence_length.append(len(temp))
           err = pd.DataFrame({"error idx":errors, "length":sentence_length})
           err.to_csv("error list.csv")
           sent = pd.DataFrame({"sentences":sentences, "label":label})
           sent.to_csv("dataset.csv",encoding="utf-8-sig",index = False)

      except:
          pass

  return pd.read_csv("dataset.csv")

In [295]:
df=read_data('/content/','라벨링170.xlsx')
df = processScript(df)
print(df.head(10))

split sentences 1 target length 1
split sentences 1 target length 1
split sentences 3 target length 3
split sentences 3 target length 3
split sentences 2 target length 2
split sentences 5 target length 5
split sentences 5 target length 4
🤗 error : length unmatched 6
LENGTH 5
split sentences 2 target length 1
🤗 error : length unmatched 7
LENGTH 2
split sentences 1 target length 1
split sentences 5 target length 5
split sentences 3 target length 3
split sentences 5 target length 6
🤗 error : length unmatched 11
LENGTH 5
split sentences 6 target length 6
split sentences 2 target length 2
split sentences 4 target length 4
split sentences 9 target length 10
🤗 error : length unmatched 15
LENGTH 9
split sentences 4 target length 4
split sentences 6 target length 7
🤗 error : length unmatched 17
LENGTH 6
split sentences 2 target length 1
🤗 error : length unmatched 18
LENGTH 2
split sentences 1 target length 1
split sentences 2 target length 2
split sentences 4 target length 4
split sentences 4 t